# FleetTrack Tech Residency 40

In [1]:
import numpy as np
import pandas as pd
import time
import json
import random

### 4 hours of data for one vehicle

In [ ]:
#initiate random seed for reproducibility
np.random.seed(42)

n = 24
timestamps = pd.date_range(start='2023-10-01 00:09:00', periods=n, freq='10min')

# Start around 30 mph and add small random changes
speeds = [30]
milage = [15000]  # starting mileage
fuel_gal = [15]  # starting fuel level in gallons
engine_temp = [190]  # starting engine temperature in Fahrenheit
mpg = 25  # miles per gallon
destination = "Location_1"

for _ in range(1, n):
    change = np.random.uniform(-5, 5)  # small change per step
    new_speed = max(0, min(70, speeds[-1] + change))  # keep in range [0,70]
    speeds.append(new_speed)
    milage.append(milage[-1] + new_speed * (10/60))  # assuming 10 min intervals
    fuel_consumption = new_speed * (10/60) / mpg
    fuel_gal.append(max(0, fuel_gal[-1] - fuel_consumption))  # prevent negative fuel
## Need condition for fuel refill
    # if fuel_gal[-1] < 2:
    #     fuel_gal[-1] = 15  # refill tank
    engine_temp.append(min(250, engine_temp[-1] + np.random.uniform(0, 5)))
## Engine temp tends to rise with speed, but cools down when speed is low

    destination = random.choice(["Location_1", "Location_2", "Location_3"]) #TBD have destination be the same for 2-4 iterations

car1 = pd.DataFrame({
    "vehicle_id": ["c1"] * n,
    "timestamp": timestamps,
    "speed_mph": np.round(speeds, 2),
    "check_engine": [0] * (n - 2) + [1, 0], #simulate a check engine event
    "milage": np.round(milage, 2),
    "fuel_gal": np.round(fuel_gal, 2),
    "engine_temp_f": np.round(engine_temp, 2),
    "destination": destination
})

car1.to_json("car1_data.json", orient="records", indent=2)


TypeError: Object of type DataFrame is not JSON serializable

### Location Database

In [ ]:
# Base coordinates roughly around Phoenix, AZ
base_lat = 33.45
base_lon = -112.07

# Slight random offsets for nearby locations (~a few miles)
def random_nearby_coordinates(base_lat, base_lon, delta=0.05):
    return (
        base_lat + random.uniform(-delta, delta),
        base_lon + random.uniform(-delta, delta)
    )

# Example fake addresses
streets = [
    "123 Main St",
    "456 Oak Ave",
    "789 Pine Rd",
    "101 Maple Blvd",
    "202 Elm St"
]

cities = ["Phoenix", "Tempe", "Scottsdale", "Mesa", "Glendale"]

# Generate 5 unique location dicts
locations = []
for i in range(5):
    lat, lon = random_nearby_coordinates(base_lat, base_lon)
    entry = {
        "location_id": f"Location_{i+1}",
        "address": {
            "street": streets[i],
            "city": cities[i],
            "state": "AZ",
            "zip": str(85000 + i)  # simple sequential ZIPs
        },
        "latitude": lat,
        "longitude": lon
    }
    locations.append(entry)

# Save to JSON file
with open("location_data.json", "w") as f:
    json.dump(locations, f, indent=2)
